In [2]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tqdm.notebook import tqdm
from pathlib import Path
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, multilabel_confusion_matrix
from sklearn.utils import shuffle

import tensorflow as tf

from importlib import reload
import sentinel_utils
import keras_model_creator
from data_generator import DataGenerator

# pd.options.mode.copy_on_write = True

In [3]:
sentinel_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12']

In [6]:
reload(sentinel_utils)

seasons = ['06']

loss = 'binary_crossentropy'
batch_size = 64
base_filters = 32
shards_dir = Path.home().joinpath('sentinel_data', 'shards')

utils = sentinel_utils.SentinelUtils(min_occurrences=20000)
selected_classes = utils.get_processed_labels()
data_summary = utils.get_data_summary(
    shards_dir, seasons, selected_classes
)

fixed_params = dict(
    seasons=seasons,
    data_summary=data_summary,
    shards_dir=shards_dir,
    loss=loss,
    batch_size=batch_size,
    base_filters=base_filters,
    dropout=0.2,
    epochs=11,
    overwrite=False,
    verbose=1
)

In [9]:
reload(keras_model_creator)

band_groups = [
    ['B2', 'B3', 'B4', 'B5'],
    ['B6', 'B7', 'B8', 'B8A'],
    ['B11', 'B12']
]

model_parent_dir = Path('models', 'band_selection')

for band_group in tqdm(band_groups):
    band_combinations = itertools.chain.from_iterable(
        itertools.combinations(band_group, r) for r in range(1, len(band_group)+1))
    for band_combination in (pbar := tqdm(list(band_combinations), leave=False)):
        pbar.set_description('-'.join(band_combination))
        band_indices = [all_bands.index(b) for b in band_combination]
    
        model_dir = model_parent_dir.joinpath(
            f'{loss}-{len(selected_classes.index)}'
            f'-{selected_classes.shape[1]}-{len(bands)}'
            f'-{"_".join(seasons)}-{batch_size}-{base_filters}'
            f'-{"_".join(band_combination)}'
        )
        model_dir.mkdir(parents=True, exist_ok=True)
    
        changing_params = dict(
            selected_classes=selected_classes,
            model_dir=model_dir,
            band_indices=band_indices,
        )
        params = fixed_params | changing_params
        model, testing_generator = keras_model_creator.KerasModelCreator(**params).run()


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

models/binary_crossentropy-237212-7-1-06-64-32-B2
models/binary_crossentropy-237212-7-1-06-64-32-B3
models/binary_crossentropy-237212-7-1-06-64-32-B4
models/binary_crossentropy-237212-7-1-06-64-32-B5
models/binary_crossentropy-237212-7-2-06-64-32-B2_B3
models/binary_crossentropy-237212-7-2-06-64-32-B2_B4
models/binary_crossentropy-237212-7-2-06-64-32-B2_B5
models/binary_crossentropy-237212-7-2-06-64-32-B3_B4
models/binary_crossentropy-237212-7-2-06-64-32-B3_B5
models/binary_crossentropy-237212-7-2-06-64-32-B4_B5
models/binary_crossentropy-237212-7-3-06-64-32-B2_B3_B4
models/binary_crossentropy-237212-7-3-06-64-32-B2_B3_B5
models/binary_crossentropy-237212-7-3-06-64-32-B2_B4_B5
models/binary_crossentropy-237212-7-3-06-64-32-B3_B4_B5
models/binary_crossentropy-237212-7-4-06-64-32-B2_B3_B4_B5


  0%|          | 0/15 [00:00<?, ?it/s]

models/binary_crossentropy-237212-7-1-06-64-32-B6
models/binary_crossentropy-237212-7-1-06-64-32-B7
models/binary_crossentropy-237212-7-1-06-64-32-B8
models/binary_crossentropy-237212-7-1-06-64-32-B8A
models/binary_crossentropy-237212-7-2-06-64-32-B6_B7
models/binary_crossentropy-237212-7-2-06-64-32-B6_B8
models/binary_crossentropy-237212-7-2-06-64-32-B6_B8A
models/binary_crossentropy-237212-7-2-06-64-32-B7_B8
models/binary_crossentropy-237212-7-2-06-64-32-B7_B8A
models/binary_crossentropy-237212-7-2-06-64-32-B8_B8A
models/binary_crossentropy-237212-7-3-06-64-32-B6_B7_B8
models/binary_crossentropy-237212-7-3-06-64-32-B6_B7_B8A
models/binary_crossentropy-237212-7-3-06-64-32-B6_B8_B8A
models/binary_crossentropy-237212-7-3-06-64-32-B7_B8_B8A
models/binary_crossentropy-237212-7-4-06-64-32-B6_B7_B8_B8A


  0%|          | 0/3 [00:00<?, ?it/s]

models/binary_crossentropy-237212-7-1-06-64-32-B11
models/binary_crossentropy-237212-7-1-06-64-32-B12
models/binary_crossentropy-237212-7-2-06-64-32-B11_B12


Evaluate the model for given years and save the results in the model's directory.

In [ ]:
all_bands = np.array(band_groups).flatten()
best_metric = 'val_weightedf1score'
best_rows = []

for band_group in tqdm(band_groups):
    band_combinations = itertools.chain.from_iterable(
        itertools.combinations(band_group, r) for r in range(1, len(band_group)+1)
    )
    for band_combination in band_combinations:
        model_dirs = list(model_parent_dir.glob(f'*-{"_".join(band_combination)}-*'))
    
        for model_dir in reversed(model_dirs):
            df = pd.read_csv(model_dir.joinpath('model.log'))
            best = df[df[best_metric] == df[best_metric].max()]
            
            best[all_bands] = [int(b in band_combination)*2 for b in all_bands]
                
            best_rows.append(best)
            
best_df = pd.concat(best_rows).round(2).reset_index(drop=True)

In [ ]:
import plot_utils
reload(plot_utils)
plot_utils.PlotUtils().line_heatmap(
    best_df, all_bands, [0.5, 0.5]
)